In [ ]:
from transformers import pipeline, AutoModel, AutoTokenizer

# model_name = 'vinai/bartpho-word-base'
model_name = 'vinai/phobert-base-v2'
classifier = pipeline("zero-shot-classification", model=model_name)

model = AutoModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)


In [ ]:
# chỉ lấy những từ có trong vocab của model, nhưng đã train model word2vec rồi.
vocab = tokenizer.get_vocab()
vocab_list = list(vocab.keys())

def filter_sentence(sentences, vocab_list):
    filtered_sentences = []
    for sentence in sentences:
        words = sentence.split()
        filtered_words = [word for word in words if word in vocab_list]
        filtered_sentences.append(' '.join(filtered_words))

    return filtered_sentences


filtered_sentence = filter_sentence(text_processed, vocab_list)
print(filtered_sentence)

In [ ]:
# some test about automodel classification.

from transformers import AutoModelForSequenceClassification

text = "Tôi là sinh viên trường đại học bách khoa hà nội"
t = classifier(text,
        candidate_labels=keyword_list,
    )
print(t)

nli_model = AutoModelForSequenceClassification.from_pretrained(model_name)

inputs = tokenizer(text, return_tensors="pt")
logits = nli_model(**inputs).logits
print('logits', logits)

In [ ]:
from torch.nn.utils.rnn import pad_sequence
print('keyword_list', keyword_list)
predictions = []
for post in filtered_sentence:
    print('post', post[:120])
    try:
        t = classifier(post,
            candidate_labels=keyword_list,
        )
        predictions.append(t)
    except Exception as e:
        print('e', e, post)
        continue

print(predictions)

def get_small_part(x):
    x['labels'] = x['labels'][0:2]
    x['scores'] = x['scores'][0:2]
    return x

predictions = list(map(get_small_part, predictions))
df_predict = pd.DataFrame(predictions, columns=["sequence", "labels", "scores"])

print(df_predict)
